In [2]:
print("OK")

OK


In [3]:
%pwd

'd:\\Chatbot-Suresh-Beekhani\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'd:\\Chatbot-Suresh-Beekhani'

In [6]:
 # For splitting text into chunks
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings  # HuggingFace embedding model for converting text to vectors
from langchain_qdrant import QdrantVectorStore  # type: ignore # For storing vectors in Qdrant # To load documents from the web (correct import statement)
import os  # Standard library for interacting with the operating system
from dotenv import load_dotenv
load_dotenv()


True

In [21]:

qdrant_api_key=os.environ.get('qdrant_api_key') 
groq_API_KEY=os.environ.get('groq_API_KEY')
qdrant_url = "https://e92b3638-9cd8-43e3-9c5d-a049f560fab2.us-east4-0.gcp.cloud.qdrant.io"
collection_name = "skchatbot"
os.environ["groq_API_KEY"] = groq_API_KEY

In [23]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_split_pdfs(folder_path, chunk_size, chunk_overlap):
    """Load PDFs from folder and split into chunks with custom separators."""
    
    # Load PDF files from the folder
    loader = DirectoryLoader(folder_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()

    # Define custom separators for splitting text
    separators = [
        "\n\n",  # Paragraph break
        "\n",    # Line break
        " ",     # Space
        ".",     # Full stop
        ",",     # Comma
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        ""        # Empty separator
    ]

    # Create a text splitter with the given chunk size, overlap, and separators
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap,
        separators=separators
    )
    
    # Split the documents into smaller chunks
    return text_splitter.split_documents(documents)


In [24]:
folder_path = "Data"  # Path to the folder containing PDF files

# Call the function with the folder path
doc_chunks = load_and_split_pdfs(folder_path, chunk_size=500, chunk_overlap=10)

In [25]:
doc_chunks

[Document(metadata={'source': 'Data\\Suresh Beekhani.pdf', 'page': 0}, page_content='Suresh Beekhani - A Passionate and Skilled Machine Learning Engineer\nSuresh Beekhani is a highly motivated and experienced Machine Learning \nEngineer with over two years of experience in developing and deploying end-to-\nend machine learning solutions. He possesses a strong foundation in data \nscience, machine learning, deep learning, and natural language processing \n(NLP). Suresh is passionate about leveraging the power of AI to extract'),
 Document(metadata={'source': 'Data\\Suresh Beekhani.pdf', 'page': 0}, page_content='meaningful insights from data and solve complex business challenges. \nKey Skills and Expertise:\nProfessional Experience:\nSuresh has a proven track record of success in various roles, including:\nProjects:\nSuresh has completed several impressive projects that demonstrate his skills \nand creativity, such as:\nMachine Learning: Proﬁcient in building predictive models, performi

In [26]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5')
    return embeddings


In [17]:
embeddings = download_hugging_face_embeddings()

C:\Users\SURESH BEEKHANI\AppData\Local\Temp\ipykernel_12676\1255207922.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5')
c:\conda_envs\mychatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [28]:
# Initialize QdrantVectorStore with documents and embedding model
qdrant = QdrantVectorStore.from_documents(
    documents=doc_chunks,              # List of Document objects to be stored in the vector store
    embedding=embeddings,              # Embedding model used to convert documents into vectors
    url=qdrant_url,                    # URL for the Qdrant service
    api_key=qdrant_api_key,            # API key for accessing the Qdrant service
    collection_name=collection_name    # Name of the collection to store the vectors in
)


In [29]:
qdrant = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    collection_name= collection_name,
    url=qdrant_url,
    api_key=qdrant_api_key
)

In [30]:
retriever = qdrant.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [31]:
retrieved_docs = retriever.invoke("Suresh beekhani ?")

In [32]:
retrieved_docs

[Document(metadata={'source': 'Data\\Suresh Beekhani.pdf', 'page': 0, '_id': '1846b96e-c7f0-4b1d-9397-5e9b3c5a85ec', '_collection_name': 'skchatbot'}, page_content='Suresh Beekhani - A Passionate and Skilled Machine Learning Engineer\nSuresh Beekhani is a highly motivated and experienced Machine Learning \nEngineer with over two years of experience in developing and deploying end-to-\nend machine learning solutions. He possesses a strong foundation in data \nscience, machine learning, deep learning, and natural language processing \n(NLP). Suresh is passionate about leveraging the power of AI to extract'),
 Document(metadata={'source': 'Data\\Suresh Beekhani.pdf', 'page': 1, '_id': 'dd064ec6-8801-4bfd-ae51-48b66240a6cc', '_collection_name': 'skchatbot'}, page_content="Education and Continuous Learning:\nSuresh holds a Bachelor's degree in Computer Science and continuously \nupdates his skills through online courses and certiﬁcations in areas like Python, \nSQL, and Data Science. \nA Co

In [33]:
print(groq_API_KEY)

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


In [34]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    api_key=groq_API_KEY,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [35]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Define a concise and informative system instruction
system_instruction = """
You are a concise and informative question-answering assistant. 
Use the provided context to answer the question accurately. 
If you don't know the answer, admit it honestly. 
Keep your response concise, ideally within three sentences.

{context}
"""

# Create a ChatPromptTemplate with the system instruction
prompt = ChatPromptTemplate.from_messages([
    ("system", system_instruction),
    ("human", "{input}")
])

In [36]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [38]:
response = rag_chain.invoke({"input": "Who is Suresh beekhan"})
print(response["answer"])

Suresh Beekhani is a highly motivated and experienced Machine Learning Engineer with over two years of experience in developing and deploying end-to-end machine learning solutions. He has a strong foundation in data science, machine learning, and natural language processing. Suresh is passionate about leveraging AI to extract meaningful insights from data and solve complex business challenges.


In [39]:
response = rag_chain.invoke({"input": "Which  service  provide suresh beekhani   ?"})
print(response["answer"])

Suresh Beekhani provides services as a Machine Learning Engineer, specifically in developing and deploying end-to-end machine learning solutions. He leverages AI to extract meaningful insights from data and solve complex business challenges. His key skills include machine learning, deep learning, and natural language processing (NLP).
